# Projeto 1 DIO

## Importação das bibliotecas

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from google.colab import files

## Carregar e preprocessar Dataset

In [ ]:
(train_data, val_data), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

In [ ]:
IMG_SIZE = (224, 224)

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

train_data = train_data.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
val_data   = val_data.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


## Transfer Learning

In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False


## Modelo Final

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)



## Treinamento

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=2
)


## Testando...

In [ ]:
# Importante definir e utilizar esta função para preparar a imagem corretamente
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


In [ ]:
uploaded = files.upload()

img_path = list(uploaded.keys())[0]  # pega o nome do arquivo enviado
img = prepare_image(img_path)

prediction = model.predict(img)[0][0]

if prediction > 0.5:
    print("🐶 Cachorro")
else:
    print("🐱 Gato")